### Data Mining in RecordOwl (Silver 1)

In [2]:
# Standard library
import os
import glob
import re
import time
import asyncio

# Third-party HTTP / async
import requests
import aiohttp
import nest_asyncio

# Data & analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Web scraping
import scrapy
from scrapy_playwright.page import PageMethod
from bs4 import BeautifulSoup

# Fuzzy matching
from fuzzywuzzy import fuzz, process

# Apify
from apify_client import ApifyClient


### Ingesting from previous layer

In [3]:
parquet_path = "./Staging/Bronze/bronze_data_1.parquet"
if os.path.exists(parquet_path):
    acra_data_filtered_by_industry = pd.read_parquet(parquet_path, engine="fastparquet")
    print(f"Loaded {len(acra_data_filtered_by_industry)} rows from {parquet_path}")
    print(acra_data_filtered_by_industry.shape)
else:
    raise FileNotFoundError(f"Parquet file not found at {parquet_path}")

Loaded 100 rows from ./Staging/Bronze/bronze_data_1.parquet
(100, 14)


In [4]:
acra_data_filtered_by_industry

,UEN,ENTITY_NAME,BUSINESS_CONSTITUTION_DESCRIPTION,ENTITY_TYPE_DESCRIPTION,ENTITY_STATUS_DESCRIPTION,REGISTRATION_INCORPORATION_DATE,PRIMARY_SSIC_CODE,SECONDARY_SSIC_CODE,STREET_NAME,POSTAL_CODE,PARENT_INDUSTRY,INDUSTRY_TYPE,SUB_INDUSTRY,DESCRIPTION
0,52895287X,CHNG LI CHERN FOOD & BEVERAGE,PARTNERSHIP,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,18/06/1999,56122,na,ANG MO KIO INDUSTRIAL PARK 2A,568047,F&B,F&B,Food Courts & Hawker Stalls,"Operators Of Food Courts, Hawker Centres, Coff..."
1,53414661M,CATZ PATISSERIE,SOLE-PROPRIETOR,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,09/06/2020,56122,na,PIONEER SECTOR 2,628390,F&B,F&B,Food Courts & Hawker Stalls,"Operators Of Food Courts, Hawker Centres, Coff..."
2,201505039Z,WANTON PTE. LTD.,None,LOCAL COMPANY,LIVE COMPANY,25/02/2015,56122,na,RACE COURSE ROAD,218699,F&B,F&B,Food Courts & Hawker Stalls,"Operators Of Food Courts, Hawker Centres, Coff..."
3,53399916B,BAR ULTRA,SOLE-PROPRIETOR,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,02/07/2019,56122,na,JALAN SULTAN,199018,F&B,F&B,Food Courts & Hawker Stalls,"Operators Of Food Courts, Hawker Centres, Coff..."
4,202429107W,OSEAS ROCK PTE. LTD.,None,LOCAL COMPANY,LIVE COMPANY,17/07/2024,56122,na,JALAN BOON LAY,619529,F&B,F&B,Food Courts & Hawker Stalls,"Operators Of Food Courts, Hawker Centres, Coff..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,T06LL0510H,POPIA MANIA LLP,None,LIMITED LIABILITY PARTNERSHIP,LIVE,15/04/2006,56140,na,UPPER SERANGOON ROAD,534530,F&B,F&B,Food Courts & Hawker Stalls,Stalls Selling Cooked Food And Prepared Drinks
96,202011894W,EIGHT FOODS PRIVATE LIMITED,None,LOCAL COMPANY,LIVE COMPANY,21/04/2020,56122,47219,PLAYFAIR ROAD,367998,F&B,F&B,Food Courts & Hawker Stalls,"Operators Of Food Courts, Hawker Centres, Coff..."
97,53478000L,JJ DON KITCHEN,SOLE-PROPRIETOR,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,17/12/2023,56122,na,TRANSIT ROAD,778897,F&B,F&B,Food Courts & Hawker Stalls,"Operators Of Food Courts, Hawker Centres, Coff..."
98,201205374H,S POWER EATING HOUSE PTE. LTD.,None,LOCAL COMPANY,LIVE COMPANY,04/03/2012,56122,46900,KITCHENER ROAD,208511,F&B,F&B,Food Courts & Hawker Stalls,"Operators Of Food Courts, Hawker Centres, Coff..."


### Mining RecordOwl

In [3]:
# COST-OPTIMIZED SCRAPER - V10 FIXED (No waitForTimeout)
from apify_client import ApifyClient
import pandas as pd
import time
import json
from requests.exceptions import RequestException
from dotenv import load_dotenv
import os

load_dotenv()

client = ApifyClient(os.getenv("APIFY_API_KEY"))

BATCH_SIZE = 800
MAX_CONCURRENCY = 5
MAX_RETRIES = 3

def create_pagefunction_v9_fixed() -> str:
    """V9: Fixed - removed waitForTimeout, using proper Puppeteer methods"""
    return """
async function pageFunction(context) {
    const { page, log, request } = context;
    const uen = request?.userData?.uen || '';

    if (!uen) return { status: 'error', uen: null, error: 'Missing UEN' };

    try {
        // Wait for search results with fallback
        try {
            await page.waitForSelector('.flex-1.min-w-0', { timeout: 20000 });
        } catch (e) {
            await page.waitForSelector('a[href*="/company/"]', { timeout: 10000 });
        }

        // Find and click link with EXACT UEN match
        const linkClickResult = await page.evaluate((targetUen) => {
            const uenUpper = targetUen.toUpperCase();
            const allLinks = document.querySelectorAll('a[href*="/company/"]');

            if (allLinks.length === 0) {
                return { clicked: false, reason: 'No company links found' };
            }

            // First pass: EXACT UEN match
            for (const link of allLinks) {
                let parent = link.parentElement;
                for (let i = 0; i < 5 && parent; i++) {
                    const parentText = (parent.innerText || parent.textContent || '').toUpperCase();
                    const uenPattern = new RegExp('\\\\b' + uenUpper.replace(/[.*+?^${}()|[\\\\]\\\\]/g, '\\\\$&') + '\\\\b');
                    
                    if (uenPattern.test(parentText)) {
                        link.click();
                        return { clicked: true, href: link.getAttribute('href'), matchType: 'exact' };
                    }
                    parent = parent.parentElement;
                }
            }

            // Fallback: Click first result
            allLinks[0].click();
            return { clicked: true, href: allLinks[0].getAttribute('href'), matchType: 'fallback' };
        }, uen);

        if (!linkClickResult.clicked) {
            return { status: 'not_found', uen, error: linkClickResult.reason || 'No search results' };
        }

        // Wait for navigation
        try {
            await page.waitForNavigation({ waitUntil: 'domcontentloaded', timeout: 25000 });
        } catch (navError) {
            const currentUrl = page.url();
            if (!currentUrl.includes('/company/')) {
                return { status: 'error', uen, error: 'Navigation failed' };
            }
        }

        // Wait for content with multiple fallback selectors
        try {
            await page.waitForSelector('dt', { timeout: 15000 });
        } catch (e1) {
            try {
                await page.waitForSelector('dd', { timeout: 10000 });
            } catch (e2) {
                try {
                    await page.waitForSelector('a[href^="mailto:"]', { timeout: 5000 });
                } catch (e3) {
                    // Continue anyway - some pages might not have these elements
                }
            }
        }

        // Verify UEN on the company page
        const uenVerification = await page.evaluate((targetUen) => {
            const uenUpper = targetUen.toUpperCase();
            const pageText = (document.body.innerText || document.body.textContent || '').toUpperCase();
            const uenPattern = new RegExp('\\\\b' + uenUpper.replace(/[.*+?^${}()|[\\\\]\\\\]/g, '\\\\$&') + '\\\\b');
            
            let uenFound = uenPattern.test(pageText);
            let uenInStructure = false;
            
            document.querySelectorAll('dt').forEach(dt => {
                const dtText = dt.textContent.toLowerCase().trim();
                if (dtText.includes('uen') || dtText.includes('registration') || dtText.includes('business registration')) {
                    const dd = dt.nextElementSibling;
                    if (dd && dd.tagName === 'DD') {
                        const ddText = dd.textContent.toUpperCase().trim();
                        if (uenPattern.test(ddText)) {
                            uenInStructure = true;
                        }
                    }
                }
            });

            return {
                uenFoundInPage: uenFound || uenInStructure,
                pageUrl: window.location.href
            };
        }, uen);

        // If UEN not found, return mismatch
        if (!uenVerification.uenFoundInPage) {
            return {
                status: 'uen_mismatch',
                uen,
                url: uenVerification.pageUrl,
                error: 'UEN not found on company page'
            };
        }

        // Extract data
        const data = await page.evaluate(() => {
            const SOCIAL_MEDIA_DOMAINS = ['facebook.com','linkedin.com','instagram.com','tiktok.com','twitter.com','x.com','youtube.com','pinterest.com'];
            
            // EMAIL
            const emails = [];
            document.querySelectorAll('a[href^="mailto:"]').forEach(a => {
                const email = a.href.replace('mailto:', '').trim();
                if (email && email.includes('@') && !emails.includes(email)) {
                    emails.push(email);
                }
            });
            
            // PHONE
            const phones = [];
            
            function formatSingaporePhone(text) {
                const digitsOnly = text.replace(/\\D/g, '');
                if (digitsOnly.length === 8) return '+65' + digitsOnly;
                if (digitsOnly.length === 10 && digitsOnly.startsWith('65')) return '+' + digitsOnly;
                if (digitsOnly.length === 11 && digitsOnly.startsWith('65')) return '+65' + digitsOnly.slice(2);
                if (digitsOnly.length >= 10) {
                    for (let i = 0; i <= digitsOnly.length - 10; i++) {
                        if (digitsOnly.slice(i, i+2) === '65' && digitsOnly.length - i >= 10) {
                            return '+' + digitsOnly.slice(i, i+10);
                        }
                    }
                }
                return null;
            }
            
            // tel: links
            document.querySelectorAll('a[href^="tel:"]').forEach(a => {
                const formatted = formatSingaporePhone(a.href.replace('tel:', '').trim());
                if (formatted && !phones.includes(formatted)) phones.push(formatted);
            });
            
            // dt/dd structure
            const phoneKeywords = ['company contact', 'business contact', 'office phone', 'main phone', 'business phone', 'company phone', 'contact number', 'phone', 'tel', 'mobile', 'call', 'contact no'];
            document.querySelectorAll('dt').forEach(dt => {
                const dtText = dt.textContent.toLowerCase().trim();
                if (phoneKeywords.some(kw => dtText.includes(kw))) {
                    const dd = dt.nextElementSibling;
                    if (dd && dd.tagName === 'DD') {
                        const formatted = formatSingaporePhone(dd.textContent.trim());
                        if (formatted && !phones.includes(formatted)) phones.push(formatted);
                    }
                }
            });
            
            // Pattern matching
            const bodyText = document.body.innerText || document.body.textContent;
            const phonePatterns = [
                /\\b(\\+65[\\s\\-]?)?([689]\\d{3}[\\s\\-]?\\d{4})\\b/g,
                /\\b65[\\s\\-]?([689]\\d{3})[\\s\\-]?(\\d{4})\\b/g,
                /\\b([689]\\d{3})[\\s\\-](\\d{4})\\b/g
            ];
            phonePatterns.forEach(pattern => {
                const matches = bodyText.matchAll(pattern);
                for (const match of matches) {
                    const formatted = formatSingaporePhone(match[0]);
                    if (formatted && !phones.includes(formatted)) phones.push(formatted);
                }
            });
            
            // WEBSITE
            const websites = [];
            document.querySelectorAll('a[href^="http"]').forEach(a => {
                const href = a.href.trim().toLowerCase();
                if (!SOCIAL_MEDIA_DOMAINS.some(d => href.includes(d)) && !href.includes('recordowl') && !href.includes('apify')) {
                    if (href.match(/\\.(com|sg|net|org|co)/)) websites.push(a.href);
                }
            });
            
            // SOCIAL MEDIA
            const facebook = [], linkedin = [], instagram = [], tiktok = [];
            document.querySelectorAll('a[href*="facebook.com"]').forEach(a => { if (!facebook.includes(a.href)) facebook.push(a.href); });
            document.querySelectorAll('a[href*="linkedin.com"]').forEach(a => { if (!linkedin.includes(a.href)) linkedin.push(a.href); });
            document.querySelectorAll('a[href*="instagram.com"]').forEach(a => { if (!instagram.includes(a.href)) instagram.push(a.href); });
            document.querySelectorAll('a[href*="tiktok.com"]').forEach(a => { if (!tiktok.includes(a.href)) tiktok.push(a.href); });
            
            // ADDRESS
            let address = null;
            const addressLabels = ['registered address', 'registered office address', 'address', 'principal place of business'];
            document.querySelectorAll('dt').forEach(dt => {
                const dtText = dt.textContent.toLowerCase().trim();
                if (addressLabels.some(label => dtText.includes(label))) {
                    const dd = dt.nextElementSibling;
                    if (dd && dd.tagName === 'DD') address = dd.textContent.trim();
                }
            });

            return {
                emails: emails.length ? emails : null,
                phones: phones.length ? phones : null,
                website: websites.length ? websites[0] : null,
                facebook: facebook.length ? facebook : null,
                linkedin: linkedin.length ? linkedin : null,
                instagram: instagram.length ? instagram : null,
                tiktok: tiktok.length ? tiktok : null,
                address: address
            };
        });

        return { status: 'success', uen, url: page.url(), ...data };

    } catch (err) {
        return { status: 'error', uen, error: err.message };
    }
}
"""

def run_scraper(client, uens):
    start_urls = [{"url": f"https://recordowl.com/search?name={uen}", "userData": {"uen": uen}} for uen in uens]

    run_input = {
    "startUrls": start_urls,
    "useChrome": False,
    "headless": True,
    "stealth": False,  # CHANGED: Removed stealth overhead
    "pageFunction": create_pagefunction_v9_fixed(),
    "maxRequestRetries": MAX_RETRIES,
    "maxRequestsPerCrawl": len(start_urls),
    "maxConcurrency": MAX_CONCURRENCY,
    "memoryMbytes": 2048,  # NEW: Reduced from 4096 MB
    "pageLoadTimeoutSecs": 25,  # CHANGED: Reduced from 40
    "pageFunctionTimeoutSecs": 60,  # CHANGED: Reduced from 120
    "waitUntil": ["domcontentloaded"],
    "proxyConfiguration": {"useApifyProxy": True},  # CHANGED: Datacenter instead of residential
}

    try:
        run = client.actor("apify/puppeteer-scraper").call(run_input=run_input)
        if not run or not isinstance(run, dict) or 'id' not in run:
            return [], "API returned invalid response"
    except Exception as e:
        return [], f"API call failed: {str(e)}"

    try:
        run_client = client.run(run["id"])
        run_info = run_client.wait_for_finish()
        status = run_info.get('status', 'UNKNOWN')

        if status in ['FAILED', 'TIMED-OUT', 'ABORTED']:
            return [], f"Actor run {status}"

        if status == "SUCCEEDED" and "defaultDatasetId" in run:
            dataset = client.dataset(run["defaultDatasetId"])
            items = list(dataset.iterate_items())
            return items, None

        return [], f"Scraping failed with status: {status}"
    except Exception as e:
        return [], f"Run monitoring error: {str(e)}"


# Execute scraper
all_results = []
total_rows = len(acra_data_filtered_by_industry)
total_batches = (total_rows + BATCH_SIZE - 1) // BATCH_SIZE

print(f"Starting scraper - {total_rows} UENs in {total_batches} batches")

for batch_idx in range(0, total_rows, BATCH_SIZE):
    batch = acra_data_filtered_by_industry.iloc[batch_idx:batch_idx + BATCH_SIZE]
    uens = [str(row['UEN']).strip() for _, row in batch.iterrows()]
    
    batch_num = (batch_idx//BATCH_SIZE)+1
    print(f"\nBatch {batch_num}/{total_batches} - {len(uens)} UENs...")
    
    items, error = run_scraper(client, uens)
    
    if error:
        print(f"  Error: {error}")
        for uen in uens:
            all_results.append({
                "UEN": uen, "Status": "error", "Error": error,
                **{k: None for k in ['Emails','Phones','Website','Facebook','LinkedIn','Instagram','TikTok','address','RecordOwl_Link']}
            })
        continue
    
    uen_map = {item.get('uen'): item for item in items if item.get('uen')}
    
    success_count = 0
    for uen in uens:
        item = uen_map.get(uen)
        if not item:
            all_results.append({
                "UEN": uen, "Status": "missing", "Error": "No data returned",
                **{k: None for k in ['Emails','Phones','Website','Facebook','LinkedIn','Instagram','TikTok','address','RecordOwl_Link']}
            })
        else:
            status = item.get('status', 'error')
            if status == 'success':
                success_count += 1
            
            all_results.append({
                'UEN': uen,
                'Status': status,
                'Error': item.get('error'),
                'Emails': item.get('emails'),
                'Phones': item.get('phones'),
                'Website': item.get('website'),
                'Facebook': item.get('facebook'),
                'LinkedIn': item.get('linkedin'),
                'Instagram': item.get('instagram'),
                'TikTok': item.get('tiktok'),
                'address': item.get('address'),
                'RecordOwl_Link': item.get('url')
            })
    
    print(f"  Success: {success_count}/{len(uens)}")
    
    if batch_num < total_batches:
        time.sleep(3)

New_Fresh_Leads = pd.DataFrame(all_results)

Starting scraper - 100 UENs in 1 batches

Batch 1/1 - 100 UENs...


[apify.puppeteer-scraper runId:jhOahoSelpbIaRoA7] -> Status: RUNNING, Message: 
[apify.puppeteer-scraper runId:jhOahoSelpbIaRoA7] -> 2026-01-27T05:19:54.642Z ACTOR: Pulling container image of build g6G5r98rF5fM6ecm3 from registry.
[apify.puppeteer-scraper runId:jhOahoSelpbIaRoA7] -> 2026-01-27T05:19:54.643Z ACTOR: Creating container.
[apify.puppeteer-scraper runId:jhOahoSelpbIaRoA7] -> 2026-01-27T05:19:54.704Z ACTOR: Starting container.
[apify.puppeteer-scraper runId:jhOahoSelpbIaRoA7] -> 2026-01-27T05:19:54.897Z Will run command: xvfb-run -a -s "-ac -screen 0 1920x1080x24+32 -nolisten tcp" /bin/sh -c ./start_xvfb_and_run_cmd.sh && npm run start:prod --silent
[apify.puppeteer-scraper runId:jhOahoSelpbIaRoA7] -> 2026-01-27T05:19:55.639Z INFO  System info {"apifyVersion":"3.4.4","apifyClientVersion":"2.16.0","crawleeVersion":"3.14.1","osType":"Linux","nodeVersion":"v22.19.0"}
[apify.puppeteer-scraper runId:jhOahoSelpbIaRoA7] -> 2026-01-27T05:19:55.792Z INFO  Configuring Puppeteer Scraper

  Success: 99/100


In [4]:
print(f"\n{'='*70}")
print(f"✅ SCRAPING COMPLETE")
print(f"{'='*70}")
print(f"📊 Results Summary:")
print(f"   • Total processed: {len(New_Fresh_Leads)}")
print(f"   • Successful: {(New_Fresh_Leads['Status']=='success').sum()}")
print(f"   • Failed: {(New_Fresh_Leads['Status']=='error').sum()}")
print(f"   • Missing: {(New_Fresh_Leads['Status']=='missing').sum()}")
print(f"\n📞 Data Extracted:")
print(f"   • Phones: {New_Fresh_Leads['Phones'].notna().sum()}")
print(f"   • Emails: {New_Fresh_Leads['Emails'].notna().sum()}")
print(f"   • Websites: {New_Fresh_Leads['Website'].notna().sum()}")
print(f"   • Facebook: {New_Fresh_Leads['Facebook'].notna().sum()}")
print(f"   • Instagram: {New_Fresh_Leads['Instagram'].notna().sum()}")
print(f"   • LinkedIn: {New_Fresh_Leads['LinkedIn'].notna().sum()}")
print(f"   • TikTok: {New_Fresh_Leads['TikTok'].notna().sum()}")
print(f"{'='*70}")

New_Fresh_Leads.head(10)


✅ SCRAPING COMPLETE
📊 Results Summary:
   • Total processed: 100
   • Successful: 99
   • Failed: 0
   • Missing: 0

📞 Data Extracted:
   • Phones: 39
   • Emails: 10
   • Websites: 30
   • Facebook: 22
   • Instagram: 20
   • LinkedIn: 2
   • TikTok: 0


,UEN,Status,Error,Emails,Phones,Website,Facebook,LinkedIn,Instagram,TikTok,address,RecordOwl_Link
0,52895287X,success,None,None,[+6564836187],https://hungrygowhere.com/singapore/chng_li_ch...,None,None,None,None,10 ANG MO KIO INDUSTRIAL PARK 2A #01-21 AMK AU...,https://recordowl.com/company/chng-li-chern-fo...
1,53414661M,success,None,None,None,https://catzpatisserie.cococart.co/,[https://www.facebook.com/CaTzPatisserie/],None,[https://www.instagram.com/catzpatisserie/],None,25 PHOENIX RISE HUA MEI GARDENS SINGAPORE 668227,https://recordowl.com/company/catz-patisserie
2,201505039Z,success,None,[sup@wantonsg.com],[+6562211336],https://wantonsg.com/,[https://www.facebook.com/wanton.sg/],None,[https://www.instagram.com/wanton.sg/?hl=en],None,458 RACE COURSE ROAD SINGAPORE 218699,https://recordowl.com/company/wanton-pte-ltd
3,53399916B,success,None,None,None,None,None,None,None,None,200 JALAN SULTAN #02-04 TEXTILE CENTRE SINGAPO...,https://recordowl.com/company/bar-ultra
4,202429107W,success,None,None,None,None,None,None,None,None,348 JALAN BOON LAY\n#01-07B\nSingapore 619529,https://recordowl.com/company/oseas-rock-pte-ltd
5,201212824G,success,None,None,[+6562550155],None,None,None,None,None,1 SOPHIA ROAD #01-52 ONE SOPHIA SINGAPORE 228149,https://recordowl.com/company/food-district-pt...
6,202103738H,success,None,None,None,https://funtoast.com.sg/,[https://www.facebook.com/funtoast],None,[https://www.instagram.com/funtoastsg/],None,12 CASHEW ROAD #01-08 ESPA SINGAPORE 679693,https://recordowl.com/company/enjour-pte-ltd
7,201334572W,success,None,[8082fnbpl@gmail.com],[+6596882525],None,[https://m.facebook.com/8082KOPITIAM/about/],None,[https://www.instagram.com/explore/locations/5...,None,21 BENOI SECTOR #01-05 MAPLETREE BENOI LOGISTI...,https://recordowl.com/company/8082-f-b-pte-ltd
8,201621449E,success,None,None,[+6562659515],https://kimlygroup.sg/,None,None,None,None,"13, WOODLANDS LINK, SINGAPORE 738725",https://recordowl.com/company/kimly-dim-sum-we...
9,202135973C,success,None,None,None,None,None,None,None,None,24A SENOKO SOUTH ROAD SELECT GROUP BUILDING 75...,https://recordowl.com/company/rong-cheng-bkt-p...


### Address Formatting 

In [5]:
# Pre-compile patterns for speed
POSTAL_RE = re.compile(r"(?:\bSingapore\b\s*)?(?P<postal>\d{6})(?!\d)", re.IGNORECASE)
UNIT_RES = [
    re.compile(r"#\s*[A-Za-z0-9]{1,4}\s*[-–]\s*[A-Za-z0-9]{1,4}", re.IGNORECASE),
    re.compile(r"\bunit\s*[#:]?\s*[A-Za-z0-9]{1,4}\s*[-–]\s*[A-Za-z0-9]{1,4}\b", re.IGNORECASE),
    re.compile(r"\bunit\s*[#:]?\s*[A-Za-z0-9]{1,5}\b", re.IGNORECASE),
]

def normalize_spaces(text: str) -> str:
    text = re.sub(r"[\n\r\t]+", " ", text)
    text = re.sub(r"\s{2,}", " ", text)
    return text.strip(" ,;|/")

def extract_postal(text: str) -> tuple[str, str | None]:
    if not text:
        return text, None
    matches = list(POSTAL_RE.finditer(text))
    if matches:
        m = matches[-1]
        postal = m.group("postal")
        start, end = m.span()
        cleaned = text[:start] + text[end:]
        cleaned = re.sub(r"\bSingapore\b", "", cleaned, flags=re.IGNORECASE)
        return normalize_spaces(cleaned), postal
    return normalize_spaces(text), None

def extract_unit(text: str) -> tuple[str, str | None]:
    if not text:
        return text, None
    for rx in UNIT_RES:
        m = rx.search(text)
        if m:
            unit_raw = m.group(0)
            cleaned = normalize_spaces(text[:m.start()] + text[m.end():])
            unit_digits = re.sub(r"^unit\s*[#:]?\s*", "", unit_raw, flags=re.IGNORECASE)
            unit_digits = normalize_spaces(unit_digits)
            unit_digits = unit_digits.replace(' – ', '-').replace('–', '-').replace(' ', '')
            unit_digits = unit_digits.lstrip('#')
            return cleaned, unit_digits
    return normalize_spaces(text), None

def clean_street(text: str) -> str | None:
    if not text:
        return None
    text = normalize_spaces(text)
    text = re.sub(r"\s*,\s*", ", ", text)
    return text if text.isupper() else text.title()

def split_address_sg(address: str) -> dict:
    if not isinstance(address, str) or not address.strip():
        return {"street": None, "unit": None, "postal_code": None, "address_clean": None}
    raw = normalize_spaces(address)
    without_postal, postal = extract_postal(raw)
    without_unit, unit = extract_unit(without_postal)
    without_unit = normalize_spaces(re.sub(r"\bSingapore\b", "", without_unit, flags=re.IGNORECASE))
    street = clean_street(without_unit)
    address_clean = normalize_spaces(" ".join(x for x in [street or "", unit or "", f"Singapore {postal}" if postal else ""] if x))
    return {"street": street, "unit": unit, "postal_code": postal, "address_clean": address_clean}

# Apply to current result DF -> create a new dataframe with clean components
if 'address' not in New_Fresh_Leads.columns:
    raise ValueError("Column 'address' not found in New_Fresh_Leads. Run the scraping cell first.")

parsed_df = pd.DataFrame(list(New_Fresh_Leads["address"].apply(split_address_sg)))

# New DataFrame with clean address fields and without raw 'address'
Cleaned_New_Fresh_Leads = New_Fresh_Leads.copy()
if 'address' in Cleaned_New_Fresh_Leads.columns:
    Cleaned_New_Fresh_Leads = Cleaned_New_Fresh_Leads.drop(columns=['address'])
Cleaned_New_Fresh_Leads["operational_street"] = parsed_df["street"]
Cleaned_New_Fresh_Leads["operational_unit"] = parsed_df["unit"]
Cleaned_New_Fresh_Leads["operational_postal_code"] = parsed_df["postal_code"]
Cleaned_New_Fresh_Leads["operational_address"] = parsed_df["address_clean"]

# Save full result to a new DataFrame and display all columns
New_Fresh_Leads_Operational = Cleaned_New_Fresh_Leads.copy()
New_Fresh_Leads_Operational

,UEN,Status,Error,Emails,Phones,Website,Facebook,LinkedIn,Instagram,TikTok,RecordOwl_Link,operational_street,operational_unit,operational_postal_code,operational_address
0,52895287X,success,None,None,[+6564836187],https://hungrygowhere.com/singapore/chng_li_ch...,None,None,None,None,https://recordowl.com/company/chng-li-chern-fo...,10 ANG MO KIO INDUSTRIAL PARK 2A AMK AUTOPOINT,01-21,568047,10 ANG MO KIO INDUSTRIAL PARK 2A AMK AUTOPOINT...
1,53414661M,success,None,None,None,https://catzpatisserie.cococart.co/,[https://www.facebook.com/CaTzPatisserie/],None,[https://www.instagram.com/catzpatisserie/],None,https://recordowl.com/company/catz-patisserie,25 PHOENIX RISE HUA MEI GARDENS,None,668227,25 PHOENIX RISE HUA MEI GARDENS Singapore 668227
2,201505039Z,success,None,[sup@wantonsg.com],[+6562211336],https://wantonsg.com/,[https://www.facebook.com/wanton.sg/],None,[https://www.instagram.com/wanton.sg/?hl=en],None,https://recordowl.com/company/wanton-pte-ltd,458 RACE COURSE ROAD,None,218699,458 RACE COURSE ROAD Singapore 218699
3,53399916B,success,None,None,None,None,None,None,None,None,https://recordowl.com/company/bar-ultra,200 JALAN SULTAN TEXTILE CENTRE,02-04,199018,200 JALAN SULTAN TEXTILE CENTRE 02-04 Singapor...
4,202429107W,success,None,None,None,None,None,None,None,None,https://recordowl.com/company/oseas-rock-pte-ltd,348 JALAN BOON LAY,01-07B,619529,348 JALAN BOON LAY 01-07B Singapore 619529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,T06LL0510H,success,None,None,None,None,None,None,None,None,https://recordowl.com/company/popia-mania-llp,510 UPPER SERANGOON ROAD SUNSHINE LODGE,04-04,534530,510 UPPER SERANGOON ROAD SUNSHINE LODGE 04-04 ...
96,202011894W,success,None,None,[+6591679396],https://www.mycareersfuture.gov.sg/job/admin/o...,None,None,None,None,https://recordowl.com/company/eight-foods-priv...,5 PEREIRA ROAD ASIAWIDE INDUSTRIAL BUILDING,04-01,368025,5 PEREIRA ROAD ASIAWIDE INDUSTRIAL BUILDING 04...
97,53478000L,success,None,None,None,https://chickenpie.sg/,None,None,None,None,https://recordowl.com/company/jj-don-kitchen,21 COMPASSVALE STREET COMPASSVALE PRIMARY SCHOOL,None,545091,21 COMPASSVALE STREET COMPASSVALE PRIMARY SCHO...
98,201205374H,success,None,None,None,None,None,None,None,None,https://recordowl.com/company/s-power-eating-h...,101 KITCHENER ROAD JALAN BESAR PLAZA,02-05,208511,101 KITCHENER ROAD JALAN BESAR PLAZA 02-05 Sin...


### Check for duplication of UEN and Phone Number

In [6]:
# Boolean masks for duplicates
uen_dup = New_Fresh_Leads_Operational["UEN"].duplicated(keep=False)

phone_dup = (
    New_Fresh_Leads_Operational["Phones"].notna() &
    New_Fresh_Leads_Operational["Phones"].duplicated(keep=False)
)

# YES/NO summary
print(
    "UEN dup:", "YES" if uen_dup.any() else "NO",
    "| Phone dup:", "YES" if phone_dup.any() else "NO"
)

# Show duplicate rows if exist
if uen_dup.any():
    print("\n🔁 Duplicate UEN rows:")
    display(New_Fresh_Leads_Operational[uen_dup])

if phone_dup.any():
    print("\n📱 Duplicate Phone rows:")
    display(New_Fresh_Leads_Operational[phone_dup])


UEN dup: NO | Phone dup: YES

📱 Duplicate Phone rows:


,UEN,Status,Error,Emails,Phones,Website,Facebook,LinkedIn,Instagram,TikTok,RecordOwl_Link,operational_street,operational_unit,operational_postal_code,operational_address
17,T16LL2399D,success,None,[ahmalormee@foodline.sg],[+6561000029],https://misstamchiak.com/ah-ma-lor-mee,[https://www.facebook.com/ahmalormee/],None,[https://www.instagram.com/ahmalormee/?hl=en],None,https://recordowl.com/company/ah-ma-lor-mee-llp,427 BUKIT PANJANG RING ROAD,08-699,670427,427 BUKIT PANJANG RING ROAD 08-699 Singapore 6...
86,53104036D,success,None,[yummeefoods@foodline.sg],[+6561000029],None,None,None,None,None,https://recordowl.com/company/yumm-ee-foods-su...,3 YUNG SHENG ROAD,03-125,618499,3 YUNG SHENG ROAD 03-125 Singapore 618499


### Drop duplicate phone numbers

In [7]:
# Convert list-like Phones into strings for comparison
New_Fresh_Leads_Operational["Phones_str"] = (
    New_Fresh_Leads_Operational["Phones"].astype(str)
)

# Create a NEW DataFrame with duplicate phone numbers removed
New_Fresh_Leads_Operational_unique_phones = (
    New_Fresh_Leads_Operational.drop_duplicates(
        subset="Phones_str", keep="first"
    )
    .drop(columns=["Phones_str"])  # clean up helper column
)

# Show size change
print("Original:", len(New_Fresh_Leads_Operational))
print("Unique Phones:", len(New_Fresh_Leads_Operational_unique_phones))


Original: 100
Unique Phones: 39


In [8]:
New_Fresh_Leads_Operational.to_parquet("./Staging/Silver/Silver_data_1.parquet", index=False, engine="fastparquet")